<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TED-Talks" data-toc-modified-id="TED-Talks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TED Talks</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Распределение-слов-по-частоте,-стоп-слова" data-toc-modified-id="Распределение-слов-по-частоте,-стоп-слова-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Распределение слов по частоте, стоп слова</a></span></li><li><span><a href="#Приведение-к-нормальной-форме" data-toc-modified-id="Приведение-к-нормальной-форме-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Приведение к нормальной форме</a></span></li><li><span><a href="#Выделение-коллокаций" data-toc-modified-id="Выделение-коллокаций-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выделение коллокаций</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TF-IDF</a></span><ul class="toc-item"><li><span><a href="#поиск-похожих" data-toc-modified-id="поиск-похожих-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>поиск похожих</a></span></li><li><span><a href="#выделение-ключевых-слов" data-toc-modified-id="выделение-ключевых-слов-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>выделение ключевых слов</a></span></li></ul></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Word2Vec</a></span><ul class="toc-item"><li><span><a href="#Sentence-Embeddings" data-toc-modified-id="Sentence-Embeddings-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Sentence Embeddings</a></span></li></ul></li><li><span><a href="#Topic-Modeling" data-toc-modified-id="Topic-Modeling-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Topic Modeling</a></span><ul class="toc-item"><li><span><a href="#Поиск-похожих-документов" data-toc-modified-id="Поиск-похожих-документов-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Поиск похожих документов</a></span></li><li><span><a href="#Краткое-описание-документов-(суммаризация)" data-toc-modified-id="Краткое-описание-документов-(суммаризация)-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Краткое описание документов (суммаризация)</a></span></li><li><span><a href="#Визуализация-тематической-модели" data-toc-modified-id="Визуализация-тематической-модели-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Визуализация тематической модели</a></span></li></ul></li></ul></div>

Uncomment if you are using colab

In [ ]:
# !pip install razdel==0.5
# !pip install pyLDAvis==2.1.2
# !pip install pymorphy2==0.8

In [ ]:
import re
from collections import Counter

import razdel
import nltk
nltk.download('stopwords')
import pymorphy2
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly
from tqdm import tqdm_notebook
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import pyLDAvis
import pyLDAvis.gensim

from IPython.display import YouTubeVideo

In [ ]:
SEED = 42
np.random.seed(SEED)

TOKEN_PATTERN = "[а-яё]+"

DATA_PATH = './data/ted_talks.csv.gz'

Uncomment if you are using colab

In [ ]:
# !mkdir ./data
# !wget https://raw.githubusercontent.com/vadim0912/MLbase_2021_spring/master/lecture06/data/ted_talks.csv.gz -O $DATA_PATH

# TED Talks

In [ ]:
df = pd.read_csv(DATA_PATH)

df.head()

In [ ]:
df.shape

In [ ]:
YouTubeVideo("ww9ClmCWBr0")

In [ ]:
df.text.str.len().hist(bins=200);

In [ ]:
min_talk_len = 750

df = df[df.text.str.len() >= min_talk_len].reset_index(drop=True)

df.text.str.len().hist(bins=200);

In [ ]:
df.shape

In [ ]:
corpus = df.text.values

# Токенизация

In [ ]:
sentence = "Как же так?! Олег... Мы же в 18.00 договаривались встретиться:("

print(sentence.split())

In [ ]:
print(re.findall("[а-яА-ЯёЁ]+", sentence))

In [ ]:
print(re.split(r"[-\s.,;!?]+", sentence))

In [ ]:
print([token.text for token in razdel.tokenize(sentence)])

In [ ]:
print(nltk.tokenize.casual_tokenize(sentence))

In [ ]:
corpus[0]

In [ ]:
def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

docs = [tokenize(text) for text in corpus]

In [ ]:
docs[0]

# Распределение слов по частоте, стоп слова

Постройте словарь `слово` -> `количество вхождений в корпус`

In [ ]:
occurence = Counter([token for doc in docs for token in doc])

In [ ]:
len(occurence)

в словаре больше 150 тыс. слов, хотя в литературном русском около 150 тыс.

In [ ]:
occurence.most_common(20)

Самые частые слова относятся к служебным частям речи и несут мало полезной информации. Их можно отбрасывать с помощью подготовленных списков

In [ ]:
stopword_set = set(nltk.corpus.stopwords.words('russian'))
# stopword_set = stopword_set.union({'это', 'который', 'весь', 'наш', 'свой', 'ещё', 'её', 'ваш', 'также', 'итак'})

In [ ]:
stopword_set

In [ ]:
max_rank = 40_000

counts = [count for _, count in occurence.most_common()[:max_rank]]

plt.plot(range(1, len(counts) + 1), counts)
plt.title('log-log scale', fontsize=18)
plt.yscale('log')
plt.xscale('log')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('rank', fontsize=16)
plt.ylabel('count', fontsize=16)
plt.tight_layout()
plt.show()

# Приведение к нормальной форме

In [ ]:
sorted_tokens = sorted(occurence.keys())

In [ ]:
sorted_tokens

в словаре встречаются разные формы одного и того же слова:
* 'аббревиатур',
* 'аббревиатура',
* 'аббревиатуре',
* 'аббревиатурой',
* 'аббревиатуру',
* 'аббревиатуры'

In [ ]:
stemmer = nltk.stem.SnowballStemmer('russian')

In [ ]:
words = [
    'аббревиатур',
    'аббревиатура',
    'аббревиатуре',
    'аббревиатурой',
    'аббревиатуру',
    'аббревиатуры',
    'человек',
    'люди',
    'людьми'
]

In [ ]:
[stemmer.stem(word) for word in words]

In [ ]:
lemmatizer = pymorphy2.MorphAnalyzer()

In [ ]:
lemmatizer.parse('стекло')[0].normal_form

In [ ]:
[lemmatizer.parse(word)[0].normal_form for word in words]

In [ ]:
lemmatizer.word_is_known('хендай')

In [ ]:
lemmatizer.parse('хендай')

In [ ]:
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm_notebook(docs)]

cleared_docs = [[token for token in text if token not in stopword_set] for text in lemmatized_docs]

In [ ]:
cleared_docs[0]

Во сколько раз уменьшился словарь после лемматизации?

In [ ]:
len({token for text in cleared_docs for token in text})

# Выделение коллокаций

Посчитайте, сколько раз пары слов встречались вместе

In [ ]:
cooccurence = Counter([(doc[i], doc[i + 1]) for doc in docs for i in range(len(doc) - 1)])

In [ ]:
cooccurence.most_common(20)

$$
PMI(x,y) = \log\frac{p\left(x,y\right)}{p\left(x\right)p\left(y\right)}
$$

$$
p(w_i) \sim \frac{\text{count}\left(w_i\right)}{\displaystyle\sum_{j=1}^n \text{count}\left( w_j \right)} = \frac{\text{count}\left(w_i\right)}{N}
$$

$$
PMI(w_1, w_2) \sim \log\left(\frac{N \cdot \text{count}\left(w_1, w_2\right)}{\text{count}\left(w_1\right) \text{count}\left(w_2\right) }\right)
$$

постройте словарь, в котором парам слов соответствует значения pmi

добавляйте в словарь только те пары слов, которые встретились в корпусе хотя бы `min_cooccur` раз

In [ ]:
cooccurence.most_common(10)

In [ ]:
occurence.most_common(10)

In [ ]:
N = sum(occurence.values())

N

In [ ]:
min_cooccur = 5

pmi = Counter()

In [ ]:
pmi.most_common()

если хочется автоматически выделять коллокации для извлечения признаков: [gensim.models.phrases.{Phrases, Phraser}](https://www.machinelearningplus.com/nlp/gensim-tutorial/#10howtocreatebigramsandtrigramsusingphrasermodels)

```python
phrases = (bows, min_count=30, progress_per=500)

bigram = gensim.models.phrases.Phraser(phrases)

bigram[bows[0]]
```

# TF-IDF

## поиск похожих

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x, lowercase=False, max_df=0.8, min_df=2, norm='l2'
).fit(cleared_docs)

получите tf-idf векторы документов, посчитайте cosine similarity каждого документа с каждым

In [ ]:
docs_tf_idf = vectorizer.transform(cleared_docs).toarray()

In [ ]:
similarities = docs_tf_idf @ docs_tf_idf.T

In [ ]:
def similar_pairs_from_matrix(similarity_matrix, threshold):
    return [
        ((i, j), similarities[i, j])
        for i in range(similarities.shape[0]) for j in range(similarities.shape[1]) 
        if j > i  and similarities[i, j] > threshold
    ]


similar_pairs = similar_pairs_from_matrix(similarities, threshold=0.7) 

In [ ]:
similar_pairs

In [ ]:
similar_pairs = sorted(similar_pairs, key=lambda x: -x[1])

In [ ]:
def print_talks(talk_ids, text_chunk=1024):

    for talk_id in talk_ids:
        record = df.loc[talk_id]

        print(record.talk)
        print('-' * 100)
        print(record.text[:text_chunk])
        print('#' * 100)

In [ ]:
for pair, score in similar_pairs:
    print("*" * 10)
    print(f"{score:.2f}")
    print("*" * 10)
    print_talks(pair)

## выделение ключевых слов

In [ ]:
top_k = 10

In [ ]:
doc_ids = [pair[0] for pair, threshold in similar_pairs]

In [ ]:
features = np.array(vectorizer.get_feature_names())

In [ ]:
[
    features[docs_tf_idf[doc_id].argsort()[::-1][:top_k]]
    for doc_id in doc_ids
]

# Word2Vec

In [ ]:
def prepare_sentence_dataset(documents):
    tokenized_sentences = []
    for document in tqdm_notebook(documents):
        for sentence in razdel.sentenize(document):
            lemmatized_tokens = [lemmatize(token) for token in tokenize(sentence.text)]
            tokenized_sentences.append(
                [token for token in lemmatized_tokens if token not in stopword_set]
            )
    return tokenized_sentences

sentence_dataset = prepare_sentence_dataset(corpus)

In [ ]:
len(sentence_dataset)

In [ ]:
sentence_dataset[0]

In [ ]:
word2vec = gensim.models.Word2Vec(
    size=100, sg=0, window=5, min_count=5, negative=20
)

In [ ]:
word2vec.build_vocab(sentence_dataset)

In [ ]:
len(word2vec.wv.vocab)

In [ ]:
word2vec.train(sentence_dataset, total_examples=word2vec.corpus_count, epochs=30)

In [ ]:
word2vec.wv.most_similar('мама')

In [ ]:
from tabulate import tabulate

test_words = ['альтруизм', 'бедность', 'платье', 'сентябрь', 'компьютер', 'лондон']

for word in test_words:
    print(word)
    print(
        tabulate(word2vec.wv.most_similar(word), tablefmt='orgtbl', headers=('neighbor', 'score')),
        end='\n\n'
    )

In [ ]:
index2word = np.array(word2vec.wv.index2word)

In [ ]:
index2word

In [ ]:
embeddings = word2vec.wv.vectors

In [ ]:
embeddings.shape

обучите TSNE на выборке случайных слов для понижения размерности векторов до двух компонент

In [ ]:
ids = np.random.randint(low=0, high=index2word.size, size=2000)

In [ ]:
embeddings_reduced = TSNE(random_state=SEED, n_components=2).fit_transform(embeddings[ids])

In [ ]:
def plot_tsne_embeddings(embeddings, annotations):

    trace = plotly.graph_objs.Scattergl(
        x = embeddings[:, 0],
        y = embeddings[:, 1],
        name = 'Embedding',
        mode = 'markers',

        marker = dict(
            colorscale='Viridis',
            size = 6,
            line = dict(width = 0.5),
            opacity=0.75
        ),
        text=annotations
    )

    layout = dict(
        title = "Word2Vec 2D TSNE Embeddings",
        yaxis = dict(zeroline = False),
        xaxis = dict(zeroline = False),
        hovermode = 'closest',
        width=800,
        height=800
    )

    return plotly.graph_objs.Figure(data=[trace], layout=layout)

In [ ]:
plot_tsne_embeddings(embeddings_reduced, index2word[ids])

## Sentence Embeddings

будем строить эмбеддинг предложения простым средним векторов слов. какие еще возможны варианты?

In [ ]:
def embed_text(text, word2index, word_embeddings):
    return np.array([
        word_embeddings[word2index[word]] for word in text 
        if word in word2index and word not in stopword_set
    ]).mean(0, keepdims=True)

In [ ]:
word2index = {word: i for i, word in enumerate(index2word)}

In [ ]:
talk2vec = np.concatenate([embed_text(doc, word2index, embeddings) for doc in cleared_docs])

In [ ]:
normed_talk2vec = normalize(talk2vec)

In [ ]:
similarities = normed_talk2vec @ normed_talk2vec.T

In [ ]:
similar_pairs = similar_pairs_from_matrix(similarities, threshold=0.96) 

In [ ]:
similar_pairs

In [ ]:
for pair, score in similar_pairs:
    print("*" * 10)
    print(f"{score:.2f}")
    print("*" * 10)
    print_talks(pair)

In [ ]:
talk2vec_reduced = TSNE(n_components=2, random_state=SEED).fit_transform(talk2vec)

plot_tsne_embeddings(talk2vec_reduced, df.talk.values)

# Topic Modeling

In [ ]:
dictionary = gensim.corpora.Dictionary(cleared_docs)

In [ ]:
len(dictionary.token2id)

In [ ]:
_ = dictionary[0] # так надо

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in cleared_docs]

In [ ]:
corpus[59]

In [ ]:
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary.id2token,
    chunksize=200,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=20,
    passes=20,
    random_state=SEED,
    per_word_topics=True,
    eval_every=None
)

In [ ]:
lda_model.print_topics()

Матрица тем:  $$\Theta \left( T \times \vert V \vert \right)$$

In [ ]:
theta = lda_model.get_topics()

In [ ]:
theta.shape

Матрица документов: $$\Phi \left(\vert D \vert \times  T \right)$$

In [ ]:
document_topics = lda_model.get_document_topics(corpus)

In [ ]:
phi = gensim.matutils.corpus2dense(document_topics, num_terms=lda_model.num_topics).T

In [ ]:
phi.shape

## Поиск похожих документов

In [ ]:
similarities = cosine_similarity(phi, phi)

similarities.min()

In [ ]:
df[df.text.apply(lambda x: 'машинное обучение' in x.lower())]

In [ ]:
doc_id = 540
top_k = 20

sorted_doc_ids = similarities[doc_id].argsort()[::-1][:top_k]

In [ ]:
pd.options.display.max_colwidth = 100

pd.DataFrame({'neighbor': df.talk.values[sorted_doc_ids], 'similarity': similarities[doc_id, sorted_doc_ids]})

## Краткое описание документов (суммаризация)

In [ ]:
def summarize(document, model, top_k):
    tokenized_sentences = []
    sentences = []
    for sentence in razdel.sentenize(document):
        sentences.append(sentence.text)
        lemmatized_tokens = [lemmatize(token) for token in tokenize(sentence.text)]
        tokenized_sentences.append(
            [token for token in lemmatized_tokens if token not in stopword_set]
        )
    
    tokenized_document = [token for sentence in tokenized_sentences for token in sentence]
    bow = dictionary.doc2bow(tokenized_document)
    document_topics = [model.get_document_topics(bow)]
    document_vector = gensim.matutils.corpus2dense(document_topics, num_terms=model.num_topics).reshape(1, -1)
    
    sentence_bows = [dictionary.doc2bow(sentence) for sentence in tokenized_sentences]
    sentence_topics = model.get_document_topics(sentence_bows)
    sentence_vectors = gensim.matutils.corpus2dense(sentence_topics, num_terms=model.num_topics).T
    
    relevance = cosine_similarity(document_vector, sentence_vectors)[0]
    most_relevant_sentences = relevance.argsort()[::-1][:top_k]
    
    return " ".join([sentence for i, sentence in enumerate(sentences) if i in most_relevant_sentences])

In [ ]:
summarize(df.text.values[540], lda_model, top_k=10)

In [ ]:
df.text.values[540]

## Визуализация тематической модели

In [ ]:
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)